# 1. SALES

In [ ]:
%%sql

# PARTIE OBLIGATOIRE

SELECT
    Year,
    Month,
    Category,
    QuantitySold,
    COALESCE(PreviousYearQuantitySold, 0) AS PreviousYearQuantitySold,
    (QuantitySold - COALESCE(PreviousYearQuantitySold, 0)) AS variation,
    CASE
        WHEN COALESCE(PreviousYearQuantitySold, 0) = 0 THEN NULL
        ELSE ( (QuantitySold - COALESCE(PreviousYearQuantitySold, 0)) / COALESCE(PreviousYearQuantitySold, 0) ) * 100
    END AS PercentageVariation
FROM (
    SELECT
        YEAR(o.orderDate) as Year,
        MONTH(o.orderDate) as Month,
        p.productLine as Category,
        SUM(od.quantityOrdered) as QuantitySold,
        (
            SELECT
                SUM(od2.quantityOrdered)
            FROM
                orders as o2
            JOIN
                orderdetails as od2 ON o2.orderNumber = od2.orderNumber
            WHERE
                YEAR(o2.orderDate) = YEAR(o.orderDate) - 1
                AND MONTH(o2.orderDate) = MONTH(o.orderDate)
                AND p.productLine = (SELECT productLine FROM products WHERE productCode = od2.productCode)
        ) as PreviousYearQuantitySold
    FROM
        orders as o
    JOIN
        orderdetails as od ON o.orderNumber = od.orderNumber
    JOIN
        products as p ON od.productCode = p.productCode
    GROUP BY
        Year, Month, Category, PreviousYearQuantitySold
) AS subquery;


# Ajouter le pourcentage

# clients qui commandent le plus
SELECT
    Country,
    c.customerName AS ClientName,
    COUNT(o.orderNumber) AS TotalOrders

FROM
    customers c
JOIN
    orders o ON c.customerNumber = o.customerNumber
GROUP BY
    ClientName, country
ORDER BY
    TotalOrders DESC

In [ ]:
%%sql

# Les 5 produits les plus vendues depuis le début de l'entreprise

SELECT
    p.productCode,
    p.productName,
    p.productLine,
    SUM(od.quantityOrdered * od.priceEach) AS ChiffreAffaires
FROM
    products p
JOIN
    orderdetails od ON p.productCode = od.productCode
GROUP BY
    p.productCode, p.productName
ORDER BY
    ChiffreAffaires DESC
LIMIT 5;

In [ ]:
%%sql

# Le chiffre d'affaire qu'a rapporté chaque produit chaque année

SELECT p.productCode, p.productName, YEAR(o.orderDate) AS Annee,
       SUM(od.priceEach * od.quantityOrdered) AS ChiffreAffaires
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
JOIN orders o ON od.orderNumber = o.orderNumber
GROUP BY p.productCode, p.productName, Annee
ORDER BY Annee, p.productCode;



In [ ]:
%%sql

# Bénéfice pour chaque produit chaque année

SELECT p.productCode, p.productName,
       YEAR(o.orderDate) AS Annee,
       SUM(od.priceEach * od.quantityOrdered) AS ChiffreAffaires,
       SUM(od.quantityOrdered) AS QuantiteVendue,
       SUM(od.priceEach * od.quantityOrdered) - (SUM(od.quantityOrdered * buyPrice)) AS Benefice
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
JOIN orders o ON od.orderNumber = o.orderNumber
GROUP BY p.productCode, p.productName, Annee
ORDER BY Annee, p.productCode;

# Autre
SELECT p.productCode, p.productName, SUM(od.quantityOrdered),
       SUM(od.priceEach * od.quantityOrdered) - (SUM(od.quantityOrdered * buyPrice)) AS Benefice
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productCode, p.productName, od.quantityOrdered
ORDER BY Benefice DESC
LIMIT 5;

In [ ]:
%%sql

SELECT
    YEAR(o.orderDate) as year,
    c.customerName,
    COUNT(o.orderNumber) AS NumberOfOrders,
    COALESCE(SUM(p.amount), 0) AS TurnoverbyCustomer,
    COALESCE(SUM(od.priceEach * od.quantityOrdered - pd.buyPrice * od.quantityOrdered), 0) AS GrossMarginByCustomer,
    COALESCE(SUM(od.priceEach * od.quantityOrdered) / COUNT(DISTINCT o.orderNumber), 0) AS AverageOrderValue
FROM customers as c
INNER JOIN payments as p ON p.customerNumber = c.customerNumber
INNER JOIN orders as o ON c.customerNumber = o.customerNumber
INNER JOIN orderdetails as od ON o.orderNumber = od.orderNumber
INNER JOIN products as pd ON pd.productCode = od.productCode
WHERE o.shippedDate IS NOT NULL
GROUP BY year, c.customerName
ORDER BY TurnoverbyCustomer DESC, c.customerName;

IndentationError: ignored

In [ ]:
%%sql

# Total Bénéfice pour chaque produit

SELECT p.productName,
       SUM(od.quantityOrdered) as TotalQuantity,
       SUM(od.priceEach * od.quantityOrdered) - (SUM(od.quantityOrdered * p.buyPrice)) AS TotalBenefice
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productName;

In [ ]:
%%sql

-- Chiffre d'Affaires par Client
SELECT
    c.customerNumber,
    c.customerName,
    COUNT(o.orderNumber) AS NumberOfOrders,
    COALESCE(SUM(p.amount), 0) AS Revenue,
    COALESCE(SUM(od.priceEach * od.quantityOrdered - p.amount), 0) AS GrossMargin,
    COALESCE(SUM(od.priceEach * od.quantityOrdered) / COUNT(DISTINCT o.orderNumber), 0) AS AverageOrderValue,
    COALESCE(AVG(DATEDIFF(p.paymentDate, o.orderDate)), 0) AS AveragePaymentDelay,
    COALESCE(SUM(p.amount) / c.creditLimit * 100, 0) AS CreditLimitUtilization
FROM customers c
LEFT JOIN payments p ON c.customerNumber = p.customerNumber
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
LEFT JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY c.customerNumber;

-- Marge Brute par Client
SELECT
    c.customerNumber,
    c.customerName,
    COALESCE(SUM(od.priceEach * od.quantityOrdered - p.amount), 0) AS GrossMargin
FROM customers c
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
LEFT JOIN orderdetails od ON o.orderNumber = od.orderNumber
LEFT JOIN payments p ON c.customerNumber = p.customerNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY c.customerNumber;

-- Nombre de Commandes par Client
SELECT
    c.customerNumber,
    c.customerName,
    COUNT(o.orderNumber) AS NumberOfOrders
FROM customers c
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY c.customerNumber;

-- Montant Moyen de la Commande par Client
SELECT
    c.customerNumber,
    c.customerName,
    COALESCE(SUM(od.priceEach * od.quantityOrdered) / COUNT(DISTINCT o.orderNumber), 0) AS AverageOrderValue
FROM customers c
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
LEFT JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY c.customerNumber;

-- Délai Moyen de Paiement par Client
SELECT
    c.customerNumber,
    c.customerName,
    COALESCE(AVG(DATEDIFF(p.paymentDate, o.orderDate)), 0) AS AveragePaymentDelay
FROM customers c
LEFT JOIN orders o ON c.customerNumber = o.customerNumber
LEFT JOIN payments p ON c.customerNumber = p.customerNumber
WHERE p.paymentDate IS NOT NULL
GROUP BY c.customerNumber, c.customerName
ORDER BY c.customerNumber;

-- Limite de Crédit Utilisée par Client
SELECT
    c.customerNumber,
    c.customerName,
    COALESCE(SUM(p.amount) / c.creditLimit * 100, 0) AS CreditLimitUtilization
FROM customers c
LEFT JOIN payments p ON c.customerNumber = p.customerNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY c.customerNumber;

# 2. FINANCES

In [ ]:
# KPI OBLIGATOIRE
%%sql

/* Chiffre d'affaire des 2 derniers mois par pays */
SELECT off.country,
       DATE_FORMAT(shippedDate, '%Y-%m') AS month_,
       SUM(priceEach * quantityOrdered) AS monthly_turnover
FROM customers AS cust
INNER JOIN orders AS o ON o.customerNumber = cust.customerNumber
INNER JOIN orderdetails AS od ON o.orderNumber = od.orderNumber
INNER JOIN employees AS emp ON emp.employeeNumber = cust.salesRepEmployeeNumber
INNER JOIN offices AS off ON off.officeCode = emp.officeCode
WHERE o.shippedDate IS NOT NULL
AND MONTH(shippedDate) >= MONTH(CURRENT_DATE) - 2
AND MONTH(shippedDate) < MONTH(CURRENT_DATE)
AND YEAR(shippedDate) = YEAR(CURRENT_DATE)
GROUP BY off.country, month_
ORDER BY off.country, month_
;

# Les commandes qui n'ont pas encore été honorées

SELECT cust.customerNumber, o.orderNumber, SUM(priceEach * quantityOrdered) AS total_per_order
FROM customers AS cust
  INNER JOIN orders AS o ON o.customerNumber = cust.customerNumber
  INNER JOIN orderdetails AS od ON o.orderNumber = od.orderNumber
  INNER JOIN (SELECT customerNumber, MAX(paymentDate) AS last_paydate
				      FROM payments
				      GROUP BY customerNumber) AS last_payments
ON last_payments.customerNumber = cust.customerNumber
WHERE status = "Shipped"
AND shippedDate > last_paydate
GROUP BY o.customerNumber, o.orderNumber;

In [ ]:
%%sql

SELECT
    c.country,
    DATE_FORMAT(o.orderDate, '%Y-%m') AS `month`,
    SUM(od.quantityOrdered * od.priceEach) AS total_revenue
FROM
    orders as o
JOIN
    orderdetails as od ON o.orderNumber = od.orderNumber
JOIN
    customers as c ON o.customerNumber = c.customerNumber
GROUP BY
    c.country, `month`
ORDER BY
    c.country, `month`;

In [ ]:
%%sql

#Rentabilité des produits

SELECT orderNumber, products.productCode, productName, quantityOrdered, priceEach, buyprice, (quantityOrdered * (priceEach-buyPrice)) AS benefits
FROM orderdetails
JOIN products ON orderdetails.productCode = products.productCode
ORDER BY benefits DESC;

# 3. LOGISTICS

In [ ]:
# KPI OBLIGATOIRE
%%sql
# the stock of the 5  most ordered product
 SELECT productLine, productName , quantityInStock, SUM(quantityOrdered) AS quantite_commande
 FROM products
 INNER JOIN orderdetails ON products.productcode = orderdetails.productcode
 GROUP BY productName,  quantityInStock, productLine
 ORDER by SUM(quantityOrdered) DESC
 LIMIT 5;

 # kpi additionnel
 # quantité par année des produits commandés
 SELECT
    p.productLine,
    p.productName,
    YEAR(o.orderDate) AS OrderYear,
    SUM(od.quantityOrdered) AS TotalQuantityOrdered
FROM
    products p
JOIN
    orderdetails od ON p.productCode = od.productCode
JOIN
    orders o ON od.orderNumber = o.orderNumber
GROUP BY
    p.productLine, p.productName, OrderYear
ORDER BY
    OrderYear, TotalQuantityOrdered DESC;

# les 5 produits les moins commandés
 SELECT productLine, productName , quantityInStock, SUM(quantityOrdered) AS quantité_commandé
 FROM products
 INNER JOIN orderdetails ON products.productcode = orderdetails.productcode
 GROUP BY productName,  quantityInStock, productLine
 ORDER by SUM(quantityOrdered) ASC
 LIMIT 5;

# Temps de délai moyen de traitement de commande
SELECT
    AVG(DATEDIFF(shippedDate, orderDate)) AS AverageProcessingTime
FROM
    orders
WHERE
    shippedDate IS NOT NULL;

# Temps de délai moyen de traitement de commande par pays du client
SELECT
    c.Country,
    round(AVG(DATEDIFF(o.shippedDate, o.orderDate)),2) AS AverageProcessingTime
FROM
    orders o
JOIN
    customers c ON o.customerNumber = c.customerNumber
WHERE
    o.shippedDate IS NOT NULL
GROUP BY
    c.Country;


# 4. RH

In [ ]:
%%sql
# KPI OBLIGATOIRE
SELECT
    CONCAT(e.firstName, ' ', e.lastName) AS EmployeeName,
    MONTH(o.orderDate) AS Lastmonth ,
    SUM(od.quantityOrdered * od.priceEach) AS MonthlyTurnover
FROM
    orders o
JOIN
    customers c ON o.customerNumber = c.customerNumber
JOIN
    employees e ON c.salesRepEmployeeNumber = e.EmployeeNumber
JOIN
    orderdetails od ON o.orderNumber = od.orderNumber
WHERE
    MONTH(o.orderDate) = MONTH(CURRENT_DATE - INTERVAL 4 MONTH)
    AND YEAR(o.orderDate) = YEAR(CURRENT_DATE - INTERVAL 4 MONTH)
GROUP BY
    EmployeeName, Lastmonth
ORDER BY
    MonthlyTurnover DESC
LIMIT 2;


In [ ]:
%%sql

## VERSION 2
SELECT
    CONCAT(e.firstName, ' ', e.lastName) AS seller_name,
    DATE_FORMAT(o.orderDate, '%Y-%m') AS month,
    SUM(od.quantityOrdered * od.priceEach) AS total_revenue,
    of.country AS Country
FROM
    orders o
JOIN
    orderdetails od ON o.orderNumber = od.orderNumber
JOIN
    customers c ON o.customerNumber = c.customerNumber
JOIN
    employees e ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN
    offices of ON of.officeCode = e.officeCode
WHERE
	o.shippedDate IS NOT NULL
GROUP BY
    seller_name, month, Country
ORDER BY
    month DESC, total_revenue DESC
LIMIT 2;


In [ ]:
%%sql

# Human Resources: Each month, the 2 sellers with the highest turnover. | Affiche pour chaque mois (Or on veut juste ceux du mois qu'on choisi)


SELECT
    DATE_FORMAT(o.orderDate, '%Y-%m') AS month,
    CONCAT(e.firstName, ' ', e.lastName) AS seller_name,
    SUM(od.priceEach * od.quantityOrdered) AS total_revenue
FROM
    orders as o
JOIN
    orderdetails as od ON o.orderNumber = od.orderNumber
JOIN
    customers as c ON o.customerNumber = c.customerNumber
JOIN
    employees as e ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY
    month, seller_name;


SELECT
    DATE_FORMAT(o.orderDate, '%Y-%m') AS month,
    CONCAT(e.firstName, ' ', e.lastName) AS seller_name,
    SUM(od.priceEach * od.quantityOrdered) AS total_revenue
FROM
    orders as o
JOIN
    orderdetails as od ON o.orderNumber = od.orderNumber
JOIN
    customers as c ON o.customerNumber = c.customerNumber
JOIN
    employees as e ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY
    month, seller_name
HAVING
    total_revenue >= (
        SELECT DISTINCT
            SUM(od1.priceEach * od1.quantityOrdered) AS monthly_revenue
        FROM
            orders o1
        JOIN
            orderdetails od1 ON o1.orderNumber = od1.orderNumber
        JOIN
            customers c1 ON o1.customerNumber = c1.customerNumber
        WHERE
            DATE_FORMAT(o1.orderDate, '%Y-%m') = month
            AND o1.shippedDate IS NOT NULL
        GROUP BY
            c1.salesRepEmployeeNumber
        ORDER BY
            monthly_revenue DESC
        LIMIT 1 OFFSET 1
    );

In [ ]:
%%sql

# Avec les mois et les années séparées

SELECT
    YEAR(o.orderDate) AS annee, -- Sépare l'année
    MONTH(o.orderDate) AS mois, -- Sépare le mois
    CONCAT(e.firstName, ' ', e.lastName) AS nom_vendeur,
    SUM(od.priceEach * od.quantityOrdered) AS total_revenue
FROM
    orders as o
JOIN
    orderdetails as od ON o.orderNumber = od.orderNumber
JOIN
    customers as c ON o.customerNumber = c.customerNumber
JOIN
    employees as e ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY
    annee, mois, nom_vendeur
HAVING
    total_revenue >= (
        SELECT DISTINCT
            SUM(od1.priceEach * od1.quantityOrdered) AS monthly_revenue
        FROM
            orders o1
        JOIN
            orderdetails od1 ON o1.orderNumber = od1.orderNumber
        JOIN
            customers c1 ON o1.customerNumber = c1.customerNumber
        WHERE
            YEAR(o1.orderDate) = annee -- Compare les années
            AND MONTH(o1.orderDate) = mois -- Compare les mois
            AND o1.shippedDate IS NOT NULL
        GROUP BY
            c1.salesRepEmployeeNumber
        ORDER BY
            monthly_revenue DESC
        LIMIT 1 OFFSET 1
    );

In [ ]:
%%sql

/* Human Resources:
Each month, the turnover of each seller
*/

SELECT
    YEAR(o.orderDate) AS year,
    MONTH(o.orderDate) AS month,
    CONCAT(e.firstName, ' ', e.lastName) AS seller_name,
    SUM(od.priceEach * od.quantityOrdered) AS total_revenue
FROM
    orders as o
JOIN
    orderdetails as od ON o.orderNumber = od.orderNumber
JOIN
    customers as c ON o.customerNumber = c.customerNumber
JOIN
    employees as e ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY
    month, seller_name;

SELECT
    YEAR(o.orderDate) AS year,
    MONTH(o.orderDate) AS month,
    CONCAT(e.firstName, ' ', e.lastName) AS seller_name,
    SUM(od.priceEach * od.quantityOrdered) AS total_revenue
FROM
    orders as o
JOIN
    orderdetails as od ON o.orderNumber = od.orderNumber
JOIN
    customers as c ON o.customerNumber = c.customerNumber
JOIN
    employees as e ON c.salesRepEmployeeNumber = e.employeeNumber
WHERE
	o.shippedDate IS NOT NULL
GROUP BY
    year, month, seller_name;

